In [1]:
import pandas as pd
import numpy as np
from numpy import nan 
import random

In [2]:
order_df = pd.read_csv('order.table.xls',header=0,sep='\t')
dna_df = pd.read_csv('DNA.table.xls',header=0,sep='\t')
delivery_df = pd.read_csv('delivery.table.xls',header=0,sep='\t')
 

In [3]:
# order_df
# dna_df
# delivery_df

,sample_code,detection_items,department,order_date,order_num,report_date,conclusion
0,ZSHC/2017/81,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,NaN,NaN
1,ZSHC/2017/80,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,NaN,NaN
2,ZSHC/2017/82,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,NaN,NaN
3,ZSHC/2017/83,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,NaN,NaN
4,ZSHC/2018/01,飞鸟残骸鉴定（常规）,杭州萧山国际机场有限公司,2018.4.9,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2022,20210429-0-014,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021-05-10 00:00:00,NaN,NaN,NaN
2023,20210429-0-015,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021-05-10 00:00:00,NaN,NaN,NaN
2024,20210429-0-016,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021-05-10 00:00:00,NaN,NaN,NaN
2025,20210429-0-017,飞鸟残骸鉴定（常规）,南宁吴圩国际机场,2021-05-10 00:00:00,NaN,NaN,NaN


In [4]:
order_df['receive_date'] = delivery_df['receive_date']

In [5]:
# order_df['receive_date'] 

0        2018.4.16
1        2018.4.16
2        2018.4.16
3        2018.4.16
4        2018.4.16
           ...    
2022    2021-05-12
2023    2021-05-12
2024    2021-05-12
2025    2021-05-12
2026    2021-05-12
Name: receive_date, Length: 2027, dtype: object

In [6]:
order_df['receive_date'] = order_df['receive_date'].str.strip(' 00:00:00').str.replace('-','.')

In [7]:
# 部分样品编码含有反斜杠，这些反斜杠在数据保存的路径被‘-’代替了 
order_df['sample_code_format'] =  order_df['sample_code'].str.replace('/','-').str.replace(' ','-').str.replace(u'加急','jiaji')

In [8]:
order_df.iloc[1958,:]

sample_code                  ZBAD2021-加急1
detection_items                飞鸟残骸鉴定（加急）
department                       北京大兴国际机场
order_date            2021-04-28 00:00:00
order_num                             NaN
report_date                           NaN
conclusion                            NaN
receive_date                    2021.04.3
sample_code_format        ZBAD2021-jiaji1
Name: 1958, dtype: object

In [9]:
# 找到那些不含有日期的字符，使他为nan：
order_df[order_df['receive_date'] == '.'] = nan
# 填充空缺的日期：
order_df['receive_date'].fillna(method='ffill',axis=0,inplace=True)



In [10]:
# order_df[order_df['receive_date'].str.contains('\b\.\b')]

# 由于日期格式不统一，所以 将字符串都转为日期类型：
order_df['receive_date'] = order_df['receive_date'].apply(pd.to_datetime,infer_datetime_format=True)


In [11]:
# order_df['receive_date'].apply(pd.to_datetime,infer_datetime_format=True)
# 再将日期转回字符串：
order_df['receive_date'] = order_df['receive_date'].apply(lambda x : x.strftime('%Y%m%d'))

In [12]:
length_df = len(order_df['receive_date'])
order_df['random_num'] = [str(random.randint(10000, 99999)) for i in range(length_df) ]

In [13]:
# order_df.random_num
order_df['order_num'] = order_df['receive_date']+ order_df['random_num'] 

# order_df['order_num'] 

0       2018041623781
1       2018041631048
2       2018041623287
3       2018041680913
4       2018041678545
            ...      
2022    2021051284874
2023    2021051224897
2024    2021051260035
2025    2021051271675
2026    2021051292958
Name: order_num, Length: 2027, dtype: object

In [14]:
# 由于有部分测序路径的文件名是以航班号而非样本名来命名，所以有必要吧航班号放进来：
order_df['event_flight']  = delivery_df['event_flight']

In [15]:
# 测试填空值：
# df = pd.DataFrame(np.random.randn(6,3))
# df.iloc[1:4,1] = nan
# df.iloc[2:5,2] = nan
# df.fillna(method='ffill',axis=0)

In [16]:
order_df['id'] = order_df.index +1 
# del order_df['random_num']
# del order_df['receive_date']
order_df.to_csv('order_df.tsv',encoding='utf_8_sig',index=False)
print('---1')

---1


In [17]:
#有些样品是没有样品名的，先挑出来，以后在处理这些样品。
order_df_null_samplecode = order_df[order_df['sample_code'].isnull()]
order_df_null_samplecode.to_csv('order_df_null_samplecode.tsv',encoding='utf_8_sig',index=False)

In [18]:

# order_df.loc[[8,9,10],'sample_code']



8     ZGGG-2018-07
9              NaN
10    ZGGG-2018-05
Name: sample_code, dtype: object

In [19]:
dna_df.to_csv('dna_df.tsv',encoding='utf_8_sig',index=False)

In [20]:
delivery_df.to_csv('delivery_df.tsv',encoding='utf_8_sig',index=False)

In [21]:
path_df = pd.read_csv('airport.csv',header=0,sep=',')

In [22]:
# path_df.head

<bound method NDFrame.head of       Unnamed: 0      date  code gene_name  \
0              0  20200427  ZSHC      cytb   
1              1  20200427  ZSHC      cytb   
2              2  20200427  ZSHC      cytb   
3              3  20200427  JYCS      cytb   
4              4  20200427  ZSHC      cytb   
...          ...       ...   ...       ...   
2581        2581  20200825    KN      cytb   
2582        2582  20200825    MF      cytb   
2583        2583  20200825    CZ      cytb   
2584        2584  20200413  JYCS      cytb   
2585        2585  20200413  JYCS     COI-1   

                                              file_name  \
0     ZSHC-2020-07_L14995-H16065_C2291.L14995.PW0050...   
1     ZSHC-2020-04_L14995-H16065_C2291.L14995.PW0050...   
2     ZSHC-2020-02_L14995-H16065_C2291.L14995.PW0050...   
3     JYCS2020-03_L14995-H16065_C2291.L14995.PW00509...   
4     ZSHC-2020-03_L14995-H16065_C2291.L14995.PW0050...   
...                                                 ...   
2581

In [23]:
result_df = pd.merge(order_df,path_df,left_on=['sample_code_format'],right_on=['sample_code'],how='outer')
result_df_event_flight = pd.merge(order_df,path_df,left_on=['event_flight'],right_on=['sample_code'],how='inner') # 

In [24]:
result_df.to_csv('result_df.tsv',encoding='utf_8_sig',index=False)
result_df_event_flight.to_csv('result_df_event_flight.tsv',encoding='utf_8_sig',index=False)

In [25]:
s = 'ZBAD2021-加急1'
s.replace('加急','jiaji')

'ZBAD2021-jiaji1'

In [26]:
# s

'ZBAD2021-加急1'

In [27]:
# order_df.iloc[[1849,1850],:]

,sample_code,detection_items,department,order_date,order_num,report_date,conclusion,receive_date,sample_code_format,random_num,event_flight,id
1849,ZGGG-2020-62,飞鸟残骸鉴定（常规）,广州白云国际机场飞行区管理部生态管理部,2020-12-20 00:00:00,2020122245946,NaN,NaN,20201222,ZGGG-2020-62,45946,ZGGG-2020-194,1850
1850,ZGGG-2020-63,飞鸟残骸鉴定（常规）,广州白云国际机场飞行区管理部生态管理部,2020-12-20 00:00:00,2020122231010,NaN,NaN,20201222,ZGGG-2020-63,31010,ZGGG-2020-202,1851


In [28]:
print('--')

--


In [29]:
# delivery_df['event_flight']

0       9C8555
1       JD0472
2       MF8026
3       JD5351
4       CA1704
         ...  
2022    HU7095
2023    CZ5776
2024    CZ5762
2025    CA1433
2026    CA1866
Name: event_flight, Length: 2027, dtype: object

In [30]:
from app import create_app
app = create_app()
app_context =  app.app_context()
app_context.push()



from app.models1 import *
from app.models import *
import random
from datetime import datetime

# 导入数据库

In [31]:
old_id = None
i = 0
for ix,val in result_df.iterrows():
    data = val.to_dict()
    id =  data.get('id')
    if not id:
        break
    if old_id == id: # 说明上一行已经保存过了，只需要将测序记录联系在这个订单上即可
        o = OrderSeq.query.filter_by(id=id).first()
#         pass
        db.session.add(o)
    else:
        o = OrderSeq()
        o.from_dict(data)
    seqData = SeqData()
    seqData.from_dict(data)
    o.seqData.append(seqData)
    db.session.add(seqData)
    
    old_id = data.get('id')
    
    i+= 1
    if i%100 == 0:
        db.session.commit()
        
db.session.commit()
print('finish ')
app_context.pop()

ArgumentError: relationship 'DNA_info' expects a class or a mapper argument (received: <class 'sqlalchemy.sql.schema.Table'>)

In [ ]:
# OrderSeq.__table__.columns.keys()